In [1]:
import xarray as xr
import fsspec
import numpy as np
import s3fs
import zarr

In [13]:
base_url = f'noaa-nws-aorc-v1-1-1km'

In [14]:
year = '1979'
single_year_url = f'{base_url}/{year}.zarr/'

In [21]:
%%time
## nonsense needed because this is apprently not able to use a FMap anymore. SMH
fs = fsspec.filesystem("s3",  anon=True,asynchronous=True)
zstore = zarr.storage.FsspecStore(fs, path=single_year_url)


ds_test = xr.open_zarr(zstore, consolidated=True)

CPU times: user 89.2 ms, sys: 13.4 ms, total: 103 ms
Wall time: 696 ms


In [22]:
ds_test

<xarray.Dataset> Size: 18TB
Dimensions:              (time: 8016, latitude: 4201, longitude: 8401)
Coordinates:
  * latitude             (latitude) float64 34kB 20.0 20.01 20.02 ... 54.99 55.0
  * longitude            (longitude) float64 67kB -130.0 -130.0 ... -60.01 -60.0
  * time                 (time) datetime64[ns] 64kB 1979-02-01 ... 1979-12-31...
Data variables:
    APCP_surface         (time, latitude, longitude) float64 2TB dask.array<chunksize=(144, 128, 256), meta=np.ndarray>
    DLWRF_surface        (time, latitude, longitude) float64 2TB dask.array<chunksize=(144, 128, 256), meta=np.ndarray>
    DSWRF_surface        (time, latitude, longitude) float64 2TB dask.array<chunksize=(144, 128, 256), meta=np.ndarray>
    PRES_surface         (time, latitude, longitude) float64 2TB dask.array<chunksize=(144, 128, 256), meta=np.ndarray>
    SPFH_2maboveground   (time, latitude, longitude) float64 2TB dask.array<chunksize=(144, 128, 256), meta=np.ndarray>
    TMP_2maboveground    (time, latitude, longitude) float64 2TB dask.array<chunksize=(144, 128, 256), meta=np.ndarray>
    UGRD_10maboveground  (time, latitude, longitude) float64 2TB dask.array<chunksize=(144, 128, 256), meta=np.ndarray>
    VGRD_10maboveground  (time, latitude, longitude) float64 2TB dask.array<chunksize=(144, 128, 256), meta=np.ndarray>

In [25]:
var='APCP_surface'

In [26]:
ds_test[var]

<xarray.DataArray 'APCP_surface' (time: 8016, latitude: 4201, longitude: 8401)> Size: 2TB
dask.array<open_dataset-APCP_surface, shape=(8016, 4201, 8401), dtype=float64, chunksize=(144, 128, 256), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 34kB 20.0 20.01 20.02 ... 54.98 54.99 55.0
  * longitude  (longitude) float64 67kB -130.0 -130.0 -130.0 ... -60.01 -60.0
  * time       (time) datetime64[ns] 64kB 1979-02-01 ... 1979-12-31T23:00:00
Attributes:
    AORC_Contact:  aorc.info@noaa.gov
    aorc_version:  v1.1
    crs:           EPSG:4326
    level:         surface
    long_name:     Total Precipitation
    short_name:    APCP_surface
    units:         kg/m^2

In [28]:
print(f'Variable size: {ds_test[var].nbytes/1e12:.1f} TB') 

Variable size: 2.3 TB


## YOU SHOULD NOT TRY OPEN THIS ON YOUR COMPUTER 
2.3 terabytes will literally take forever to load. 
Instead, what you should do is something like the following: 
1) identify all of the points that you want to find (to do this, I will be using the excel file that you already provided me)
2) Parse down the data to only the cells you want (or region you want, but in this case you want points)
3) Save these files in a per year format (e.g. one file for every year). These can be useful to keep around, but step 4 makes it more agreeable to workflows
4) Concat all of the files together and resave to a single netcdf (or whatever file you want, but netcdf is usually my go to). 

In [33]:
## in this case, the ZARR data is nice and has latitiude and longitude loaded in as keys, so we just need to do the following lines:
import pandas as pd

data_locations = pd.read_csv('./largest_cities50.csv')
data_locations = data_locations[:51]
data_locations

,filename,lat,long,lat_long
0,New__York,40.66000,-73.940000,"40.66, -73.94"
1,Los_Angeles,34.02000,-118.410000,"34.02, -118.41"
2,Chicago,41.84000,-87.680000,"41.84, -87.68"
3,Houston,29.79000,-95.390000,"29.79, -95.39"
4,Phoenix,33.57000,-112.090000,"33.57, -112.09"
5,Philadelphia,40.01000,-75.130000,"40.01, -75.13"
6,San_Antonio,29.46000,-98.520000,"29.46, -98.52"
7,San_Diego,32.81000,-117.140000,"32.81, -117.14"
8,Dallas,32.79000,-96.770000,"32.79, -96.77"
9,Jacksonville,30.34000,-81.660000,"30.34, -81.66"


In [36]:
x_xarray = xr.DataArray(data_locations.long.values, dims=['location'])
y_yarray = xr.DataArray(data_locations.lat.values, dims=['location'])

test__extract = ds_test[var].sel(longitude=x_xarray, latitude=y_yarray,method='nearest')

In [37]:
test__extract

<xarray.DataArray 'APCP_surface' (time: 8016, location: 51)> Size: 3MB
dask.array<transpose, shape=(8016, 51), dtype=float64, chunksize=(144, 51), chunktype=numpy.ndarray>
Coordinates:
    latitude   (location) float64 408B 40.66 34.02 41.84 ... 27.97 32.7 39.12
    longitude  (location) float64 408B -73.94 -118.4 -87.68 ... -97.12 -75.47
  * time       (time) datetime64[ns] 64kB 1979-02-01 ... 1979-12-31T23:00:00
Dimensions without coordinates: location
Attributes:
    AORC_Contact:  aorc.info@noaa.gov
    aorc_version:  v1.1
    crs:           EPSG:4326
    level:         surface
    long_name:     Total Precipitation
    short_name:    APCP_surface
    units:         kg/m^2

In [42]:
## now let's check to see how big this is (its way smaller I promise)
print(f'Variable size: {test__extract.nbytes/1e6:.5f} MB') 

Variable size: 3.27053 MB


In [61]:
test__extract =test__extract.assign_coords(location=data_locations.filename.astype('unicode'))
test__extract = test__extract.rename('RAINRATE')

In [74]:
import dask
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 7
Total threads: 14,Total memory: 36.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:62864,Workers: 7
Dashboard: http://127.0.0.1:8787/status,Total threads: 14
Started: Just now,Total memory: 36.00 GiB
Comm: tcp://127.0.0.1:62883,Total threads: 2
Dashboard: http://127.0.0.1:62891/status,Memory: 5.14 GiB
Nanny: tcp://127.0.0.1:62867,


In [62]:
dask.config.set(temporary_directory='./test-directory/') # this is for all of our workers to do their thing
filename=f'./AORC-1km-data/AORC_1km_50largest_cities_1979.nc'


In [63]:
test__extract.to_netcdf(filename, 'w') # this works, now to make our for loop

In [64]:
## check to make sure the file we wrote looks like what we want it to 
test = xr.open_dataset(filename)
test ## yay!! this all looks good, now lets loop over all of these!

<xarray.Dataset> Size: 3MB
Dimensions:    (location: 51, time: 8016)
Coordinates:
    latitude   (location) float64 408B ...
    longitude  (location) float64 408B ...
  * time       (time) datetime64[ns] 64kB 1979-02-01 ... 1979-12-31T23:00:00
  * location   (location) <U16 3kB 'New__York' 'Los_Angeles' ... 'Dover'
Data variables:
    RAINRATE   (time, location) float64 3MB ...

In [68]:
from tqdm import tqdm

## To loop over this in a clean way, we just do all of the stuff we did, but just wrapped in a for loop
1) create the dates we want to loop over
2) iterate over those dates
3) create the URL we need to go find
4) create the map of the zarr files we need
5) create new xarray datasets so that we can load in locations

In [76]:
dates = pd.date_range(start='01-01-2012',end='01-01-2024',freq='YE')
for year in tqdm(dates):
    # create a year string
    year = str(year.year)
    single_year_url = f'{base_url}/{year}.zarr/'

    # create a map and a zstore for the specifc year we want
    fs = fsspec.filesystem("s3",  anon=True,asynchronous=True)
    zstore = zarr.storage.FsspecStore(fs, path=single_year_url)


    ds_test = xr.open_zarr(zstore, consolidated=True)


    x_xarray = xr.DataArray(data_locations.long.values, dims=['location'])
    y_yarray = xr.DataArray(data_locations.lat.values, dims=['location'])

    test__extract = ds_test[var].sel(longitude=x_xarray, latitude=y_yarray,method='nearest')

    test__extract =test__extract.assign_coords(location=data_locations.filename.astype('unicode'))
    test__extract = test__extract.rename('RAINRATE')

    filename=f'./AORC-1km-data/AORC_1km_50largest_cities_{year}.nc'
    test__extract.to_netcdf(filename, 'w') # this works, now to make our for loop

100%|██████████| 12/12 [06:11<00:00, 30.95s/it]


## and that is how we loop through terabytes of data and consolidate it down to just a few files without blowing up our CPU

## Finally, because we want to usually work with a single file, we can do the following: 


In [80]:
all_data_concat = xr.open_mfdataset('./AORC-1km-data/*')
all_data_concat
all_data_concat.to_netcdf('./AORC-1km-data-allyears.nc')

In [77]:
client.close()

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x12e28f310>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1306a21d0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x12abc7a10>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x11dd2a7d0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x11f60c310>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1306f2d10>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x16bde3a10>
